## Project root and imports

In [1]:
import sys
from pathlib import Path

# Assume notebook lives in PROJECT_ROOT / "notebooks"
project_root = Path.cwd().parent
print("Detected project root:", project_root)

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print("Added to sys.path.")

# Now import your pipeline
from vector_pipeline.settings import DATA_PATH, CHROMA_DIR, COMBINED_TEXT_COLUMN
from vector_pipeline.ingestion import build_or_load_vectorstore
from vector_pipeline.retrieval import (
    load_vectorstore,
    retrieve_documents,
    retrieve_products,
    summarize_retrieved_products,
    rag_answer,
    print_product_card,
    preview_vectorstore,
    debug_similarity_search_with_scores,
    debug_single_retrieval,
    debug_single_rag,
    interactive_retrieval_chat,
    interactive_rag_chat,
)


Detected project root: /home/viktoriia/Documents/ML/project-25-2-scrum-team-data
Added to sys.path.


`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


## Check data and paths

In [2]:
import pandas as pd

print("DATA_PATH:", DATA_PATH.resolve(), "-> exists:", DATA_PATH.exists())
print("CHROMA_DIR:", CHROMA_DIR.resolve(), "-> exists:", CHROMA_DIR.exists())

assert DATA_PATH.exists(), "processed_product.pkl is missing. Run data_prep first."

df = pd.read_pickle(DATA_PATH)
print("Rows:", len(df))
print("Columns:", list(df.columns))

# Quick look at combined_text length
df["combined_length"] = df[COMBINED_TEXT_COLUMN].fillna("").astype(str).str.len()
df[["product_id", COMBINED_TEXT_COLUMN, "combined_length"]].head()


DATA_PATH: /home/viktoriia/Documents/ML/project-25-2-scrum-team-data/data/processed_product.pkl -> exists: True
CHROMA_DIR: /home/viktoriia/Documents/ML/project-25-2-scrum-team-data/chroma_db -> exists: True
Rows: 3734
Columns: ['rating', 'rating_count', 'price', 'store', 'product_id', 'color', 'combined_text']


,product_id,combined_text,combined_length
0,B0BSGM6CQ9,Product Title: Ernie Ball Mondo Slinky Nickelw...,1532
1,B09W4F2X6S,Product Title: BONAOK Wireless Bluetooth Karao...,1515
2,B076B8G5D8,Product Title: Boya BYM1 by Shotgun Video Micr...,1506
3,B0BTC9YJ2W,Product Title: D'Addario Guitar Strings - Phos...,1792
4,B018FCZKR2,Product Title: Amazon Basics Adjustable Guitar...,820


## Build or load vectorstore (ingestion test)

In [3]:
vectorstore = build_or_load_vectorstore()
print("Vectorstore ready ✅")
print("Persist directory:", CHROMA_DIR.resolve())

Loading existing Chroma DB from /home/viktoriia/Documents/ML/project-25-2-scrum-team-data/chroma_db ...
Vectorstore loaded successfully.
Document count: 6401
Vectorstore ready ✅
Persist directory: /home/viktoriia/Documents/ML/project-25-2-scrum-team-data/chroma_db


## Load vectorstore via retrieval module

In [4]:
vs = load_vectorstore()
print("Loaded vectorstore from retrieval.load_vectorstore() ✅")

Loaded vectorstore from retrieval.load_vectorstore() ✅


## Preview DB

In [5]:
preview_vectorstore(vs, k=5)

Got 5 documents from vectorstore.

=== Doc 1 ===
Metadata:
  start_index: 1827
  product_id: B09XB69QQ4
  rating: 4.7
  rating_count: 5399
  row_index: 269
  store: Donner
  price: 7.19
  color: Dazzle pattern

Text snippet:
  Description:

----------------------------------------------------------------------

=== Doc 2 ===
Metadata:
  row_index: 871
  product_id: B07DD9KCX4
  rating: 4.1
  price: 179.99
  store: TOSING
  start_index: 1935
  color: Gold
  rating_count: 1686

Text snippet:
  Description:

----------------------------------------------------------------------

=== Doc 3 ===
Metadata:
  product_id: B0073J2VBC
  start_index: 2275
  row_index: 1704
  rating_count: 1065
  price: 11.99
  color: Not Specified
  rating: 4.7
  store: PWR+

Text snippet:
  Description:

----------------------------------------------------------------------

=== Doc 4 ===
Metadata:
  rating_count: 940
  rating: 4.6
  start_index: 2315
  product_id: B07TM115DZ
  row_index: 2027
  store: Bim-Bam-Bo

## Choose a test query

In [6]:
TEST_QUERY_RETRIEVAL = "wireless headphones with good battery life"
TEST_QUERY_RAG = "Which wireless headphones have the best battery life?"

## Test retrieve_documents (no reranker)

In [7]:
docs_no_rerank = retrieve_documents(
    query=TEST_QUERY_RETRIEVAL,
    vs=vs,
    k=4,
    use_reranker=False,
)

print(f"Retrieved {len(docs_no_rerank)} docs WITHOUT reranker\n")
for i, doc in enumerate(docs_no_rerank, start=1):
    print(f"--- Doc {i} ---")
    print("Metadata:", doc.metadata)
    print("Snippet:", doc.page_content[:300].replace("\n", " "))
    print()


Retrieved 4 docs WITHOUT reranker

--- Doc 1 ---
Metadata: {'store': 'Pyle', 'rating_count': 4723, 'price': 74.99, 'rating': 4.2, 'product_id': 'B07SPHSDRV', 'color': 'Black', 'row_index': 329, 'start_index': 2878}
Snippet: Bluetooth for Wireless Music Streaming: Built-in Bluetooth for Wireless Music Streaming: Simple & Hassle-Free Pairing: Simple & Hassle-Free Pairing: Works with All of Today’s Latest Devices: Works with All of Today’s Latest Devices: Smartphones, Tablets, Laptops, Computers, etc.: Smartphones, Tablet

--- Doc 2 ---
Metadata: {'product_id': 'B015NBUYO2', 'start_index': 4323, 'row_index': 1487, 'price': 176.91, 'store': 'Pyle', 'rating_count': 1176, 'color': 'Black', 'rating': 4.1}
Snippet: Mic Operation Range: 100 ft.: Wireless Mic Operation Range: 100 ft.: System Battery Life: Approx. 10 Hours: System Battery Life: Approx. 10 Hours: Average Charging Time: 8 - 10 Hours: Average Charging Time: 8 - 10 Hours: Battery Type: Lead Acid, 12V (4.5A): Battery Type: Lead Acid, 

## Test retrieve_documents (with reranker)

In [8]:
docs_with_rerank = retrieve_documents(
    query=TEST_QUERY_RETRIEVAL,
    vs=vs,
    k=4,
    use_reranker=True,
)

print(f"Retrieved {len(docs_with_rerank)} docs WITH reranker\n")
for i, doc in enumerate(docs_with_rerank, start=1):
    print(f"--- Doc {i} ---")
    print("Metadata:", doc.metadata)
    print("Snippet:", doc.page_content[:300].replace("\n", " "))
    print()

Retrieved 4 docs WITH reranker

--- Doc 1 ---
Metadata: {'product_id': 'B0814PFYYX', 'color': '2.4G Wireless Headset Microphones', 'start_index': 330, 'price': 39.99, 'row_index': 2616, 'store': '1Mii', 'rating_count': 756, 'rating': 4.2}
Snippet: Features: [2.4G CONNECTION]1Mii headset with microphone is engineered with the most advanced 2.4G wireless technology. The wireless headset microphone automatically connects to when power on.With low radiation, low interference, high signal stability, fast transmission, short audio delay. The freque

--- Doc 2 ---
Metadata: {'price': 29.88, 'start_index': 294, 'color': 'U12AS', 'store': 'XIAOKOA', 'rating_count': 1253, 'product_id': 'B09XF54V38', 'row_index': 1355, 'rating': 4.0}
Snippet: Features: 🎤Upgraded Head Microphone: wireless microphone headset with LED Display,it can be clearly displayed the frequency,volume and battery status.▶Up to 165 feet transmission range in an open areal.▶Support 40 sets to use at the same time.🔔Not Support Mo

## Test print_product_card

In [9]:
if docs_with_rerank:
    test_doc = docs_with_rerank[0]
    print("Testing print_product_card() on one doc\n")
    print_product_card(test_doc)
else:
    print("No docs retrieved; cannot test print_product_card.")

Testing print_product_card() on one doc

──────────────────────────────────────────────
🛍️  Product B0814PFYYX (2.4G Wireless Headset Microphones)
   Store: 1Mii
   Price: 39.99
   Rating: 4.2 ⭐ (756 reviews)
   Description snippet:
   Features: [2.4G CONNECTION]1Mii headset with microphone is engineered with the most advanced 2.4G wireless technology. The wireless headset microphone automatically connects to when power on.With low radiation, low interference, high signal stability
──────────────────────────────────────────────


## Test debug_similarity_search_with_scores

In [10]:
print("Testing debug_similarity_search_with_scores()\n")
debug_similarity_search_with_scores(
    vs=vs,
    query=TEST_QUERY_RETRIEVAL,
    k=5,
)

Testing debug_similarity_search_with_scores()

Got 5 results for query: 'wireless headphones with good battery life'

=== Result 1 ===  (score=0.3188)
──────────────────────────────────────────────
🛍️  Product B07SPHSDRV (Black)
   Store: Pyle
   Price: 74.99
   Rating: 4.2 ⭐ (4723 reviews)
   Description snippet:
   Bluetooth for Wireless Music Streaming: Built-in Bluetooth for Wireless Music Streaming: Simple & Hassle-Free Pairing: Simple & Hassle-Free Pairing: Works with All of Today’s Latest Devices: Works with All of Today’s Latest Devices: Smartphones, Tabl
──────────────────────────────────────────────
=== Result 2 ===  (score=0.3311)
──────────────────────────────────────────────
🛍️  Product B015NBUYO2 (Black)
   Store: Pyle
   Price: 176.91
   Rating: 4.1 ⭐ (1176 reviews)
   Description snippet:
   Mic Operation Range: 100 ft.: Wireless Mic Operation Range: 100 ft.: System Battery Life: Approx. 10 Hours: System Battery Life: Approx. 10 Hours: Average Charging Time: 8 - 10 Hour

## Test debug_single_retrieval

In [11]:
print("Testing debug_single_retrieval()\n")
debug_single_retrieval(
    vs=vs,
    query=TEST_QUERY_RETRIEVAL,
    k=4,
    use_reranker=True,
)

Testing debug_single_retrieval()


Retrieved 4 documents for query: 'wireless headphones with good battery life'
Reranker enabled: True

──────────────────────────────────────────────
🛍️  Product B0814PFYYX (2.4G Wireless Headset Microphones)
   Store: 1Mii
   Price: 39.99
   Rating: 4.2 ⭐ (756 reviews)
   Description snippet:
   Features: [2.4G CONNECTION]1Mii headset with microphone is engineered with the most advanced 2.4G wireless technology. The wireless headset microphone automatically connects to when power on.With low radiation, low interference, high signal stability
──────────────────────────────────────────────
──────────────────────────────────────────────
🛍️  Product B09XF54V38 (U12AS)
   Store: XIAOKOA
   Price: 29.88
   Rating: 4.0 ⭐ (1253 reviews)
   Description snippet:
   Features: 🎤Upgraded Head Microphone: wireless microphone headset with LED Display,it can be clearly displayed the frequency,volume and battery status.▶Up to 165 feet transmission range in an open are

## Test retrieve_products and summarize_retrieved_products

In [16]:
print("Testing retrieve_products()\n")
products = retrieve_products(
    query=TEST_QUERY_RETRIEVAL,
    k=3,
    use_reranker=True,
)

for i, p in enumerate(products, start=1):
    print(f"=== Product {i} ===")
    print("Metadata:", p["metadata"])
    print("Snippet:", p["snippet"])
    print()

print("Testing summarize_retrieved_products()\n")
summary_text = summarize_retrieved_products(
    query=TEST_QUERY_RETRIEVAL,
    k=3,
    use_reranker=True,
)
print(summary_text)

Testing retrieve_products()

=== Product 1 ===
Metadata: {'row_index': 2616, 'color': '2.4G Wireless Headset Microphones', 'product_id': 'B0814PFYYX', 'rating_count': 756, 'store': '1Mii', 'start_index': 330, 'price': 39.99, 'rating': 4.2}
Snippet: Features: [2.4G CONNECTION]1Mii headset with microphone is engineered with the most advanced 2.4G wireless technology. The wireless headset microphone automatically connects to when power on.With low radiation, low interference, high signal stability, fast transmission, short audio delay. The frequency of the wireless mic provides confidentiality, anti-interference ability and loud, clear, high fi

=== Product 2 ===
Metadata: {'color': 'U12AS', 'row_index': 1355, 'rating_count': 1253, 'start_index': 294, 'product_id': 'B09XF54V38', 'price': 29.88, 'rating': 4.0, 'store': 'XIAOKOA'}
Snippet: Features: 🎤Upgraded Head Microphone: wireless microphone headset with LED Display,it can be clearly displayed the frequency,volume and battery status.▶Up

## Test rag_answer

In [13]:
print("Testing rag_answer()\n")
answer = rag_answer(
    query=TEST_QUERY_RAG,
    vs=vs,
    k=4,
    use_reranker=True,
)
print("RAG answer:\n")
print(answer)

Testing rag_answer()



This is a friendly reminder - the current text generation call has exceeded the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


RAG answer:

Use ONLY the following product information to answer the question.
If the answer is not in the context, say you don't know.

CONTEXT:
Bluetooth for Wireless Music Streaming: Built-in Bluetooth for Wireless Music Streaming: Simple & Hassle-Free Pairing: Simple & Hassle-Free Pairing: Works with All of Today’s Latest Devices: Works with All of Today’s Latest Devices: Smartphones, Tablets, Laptops, Computers, etc.: Smartphones, Tablets, Laptops, Computers, etc.: Bluetooth Version: 4.0: Bluetooth Version: 4.0: Bluetooth Network Name: ‘PYLEUSA’: Bluetooth Network Name: ‘PYLEUSA’: Wireless Range: Up to 32’ ft.: Wireless Range: Up to 32’ ft.: What’s in the Box:: PA Speaker System: PA Speaker System: Remote Control: Remote Control: Power Adapter: Power Adapter: Technical Specs:: Power Output: 400 Watt MAX: Power Output: 400 Watt MAX: Sound System: 8’’ -inch Subwoofer: Sound System: 8’’ -inch Subwoofer: Impedance: 4 Ohm: Impedance: 4 Ohm: Frequency Response: 55-20kHz: Frequency Resp

## Test debug_single_rag

In [14]:
print("Testing debug_single_rag()\n")
debug_single_rag(
    vs=vs,
    query=TEST_QUERY_RAG,
    k=4,
    use_reranker=True,
)

Testing debug_single_rag()

RAG answer for query: 'Which wireless headphones have the best battery life?'  (reranker=True)

Use ONLY the following product information to answer the question.
If the answer is not in the context, say you don't know.

CONTEXT:
Bluetooth for Wireless Music Streaming: Built-in Bluetooth for Wireless Music Streaming: Simple & Hassle-Free Pairing: Simple & Hassle-Free Pairing: Works with All of Today’s Latest Devices: Works with All of Today’s Latest Devices: Smartphones, Tablets, Laptops, Computers, etc.: Smartphones, Tablets, Laptops, Computers, etc.: Bluetooth Version: 4.0: Bluetooth Version: 4.0: Bluetooth Network Name: ‘PYLEUSA’: Bluetooth Network Name: ‘PYLEUSA’: Wireless Range: Up to 32’ ft.: Wireless Range: Up to 32’ ft.: What’s in the Box:: PA Speaker System: PA Speaker System: Remote Control: Remote Control: Power Adapter: Power Adapter: Technical Specs:: Power Output: 400 Watt MAX: Power Output: 400 Watt MAX: Sound System: 8’’ -inch Subwoofer: Soun

## 

## To rebuild Chroma vectorstore

In [15]:
# ⚠️ Only run this cell when you REALLY want to rebuild the DB
#     (e.g., when processed_product.pkl has changed).
from vector_pipeline.ingestion import build_chroma_vectorstore

REBUILD = False  # set to True when you intentionally rebuild

if REBUILD:
    print("Rebuilding Chroma DB from scratch...")
    new_vs = build_chroma_vectorstore()
    print("New DB built. Document count:", new_vs._collection.count())

    # Zip the directory for sharing (same style as Muhammet used)
    from vector_pipeline.settings import PROJECT_ROOT

    zip_base = PROJECT_ROOT / "chroma_db"  # base name without .zip
    print(f"Creating archive at: {zip_base}.zip")

    shutil.make_archive(
        base_name=str(zip_base),
        format="zip",
        root_dir=str(CHROMA_DIR),
    )

    print("✅ New chroma_db.zip created.")
else:
    print("REBUILD is False – skipping DB rebuild & re-zip.")


REBUILD is False – skipping DB rebuild & re-zip.
